In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_json("data\\Sarcasm_Headlines_Dataset_v2.json", lines=True)
df["sentence_length"] = df["headline"].str.split(" ").apply(len)
df = df.drop_duplicates()

# Split DataFrame into train/test
# TODO: split test into validation and test sets, then replace validation set accordingly in the model training
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df  = train_test_split(temp_df, test_size=0.5, random_state=42)

# X_train, X_temp, y_train, y_temp = train_test_split(df['headline'], df['is_sarcastic'], test_size=0.2, random_state=69)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=69)

In [2]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is enabled
print(torch.version.cuda)  # Should print the CUDA version if available

True
12.1


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge-mnli")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v2-xlarge-mnli")

In [8]:
# model_inputs = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors="pt").to("cuda")

In [7]:
from datasets import Dataset,DatasetDict

# Convert to Hugging Face DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "val": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["headline"])
dataset = dataset.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/22893 [00:00<?, ? examples/s]

Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

In [10]:
print(dataset.column_names)
dataset = dataset.remove_columns(['article_link', 'sentence_length', '__index_level_0__', 'headline'])
dataset = dataset.rename_column("is_sarcastic", "label")
print(dataset.column_names)

{'train': ['is_sarcastic', 'headline', 'article_link', 'sentence_length', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'], 'val': ['is_sarcastic', 'headline', 'article_link', 'sentence_length', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['is_sarcastic', 'headline', 'article_link', 'sentence_length', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask']}
{'train': ['label', 'input_ids', 'token_type_ids', 'attention_mask'], 'val': ['label', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['label', 'input_ids', 'token_type_ids', 'attention_mask']}


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="D:/deberta-v2-xlarge-mnli-finetuned-sarcasm",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    push_to_hub=True,
)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

C:\Users\Raygu\AppData\Local\Temp\ipykernel_22992\1167372532.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.395000
1000,0.347700
1500,0.363100
2000,0.348600
2500,0.289900
3000,0.244200
3500,0.169700
4000,0.146700
4500,0.168600
5000,0.173500


TrainOutput(global_step=5724, training_loss=0.2501153332798903, metrics={'train_runtime': 3061.1394, 'train_samples_per_second': 14.957, 'train_steps_per_second': 1.87, 'total_flos': 3984117431788260.0, 'train_loss': 0.2501153332798903, 'epoch': 2.0})

In [ ]:
inputs = tokenizer(test_df["headline"].tolist(), padding=True, truncation=True, return_tensors="pt").to("cuda")  
predictions = trainer.predict(inputs)
predicted_labels = torch.argmax(predictions.predictions, axis=-1)
print(predicted_labels)

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("charaydes/deberta-v2-xlarge-mnli-finetuned-sarcasm")
inputs = tokenizer(test_df["headline"].tolist(), padding=True, truncation=True, return_tensors="pt").to("cuda") 

In [54]:
from torch.utils.data import DataLoader, TensorDataset
# Create a DataLoader to batch inputs
batch_size = 16  # or even smaller if needed (like 8)
dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"])
loader = DataLoader(dataset, batch_size=batch_size)

In [47]:
print(model.forward)

<bound method DebertaV2ForSequenceClassification.forward of DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1536, padding_idx=0)
      (LayerNorm): LayerNorm((1536,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1536, out_features=1536, bias=True)
              (key_proj): Linear(in_features=1536, out_features=1536, bias=True)
              (value_proj): Linear(in_features=1536, out_features=1536, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_feature

In [55]:
all_preds = []

In [56]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("charaydes/deberta-v2-xlarge-mnli-finetuned-sarcasm").to("cuda") 
torch.cuda.empty_cache()
with torch.no_grad():
    for batch in loader:
        input_ids, attention_mask = [x.to("cuda") for x in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())

In [57]:
print(all_preds)


[0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 

In [59]:
print(test_df["is_sarcastic"].tolist())
labels = test_df["is_sarcastic"].tolist()

[0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 

In [62]:
from sklearn.metrics import classification_report

print(classification_report(labels, all_preds))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      1484
           1       0.97      0.92      0.94      1378

    accuracy                           0.95      2862
   macro avg       0.95      0.95      0.95      2862
weighted avg       0.95      0.95      0.95      2862

